In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
from urllib.request import Request, urlopen

In [6]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [7]:
def scrape_links(page_number):
    url = f"https://jabar.antaranews.com/jabar-terkini/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [8]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 9 links from page 1
Total Links: 9


In [9]:
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "tags-wrapper"})
            if author_elem:
                author_text = author_elem.find('span')
                author_text = author_text.text.split(':')[1]
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix"})
            
            if body_elem:
                content_elem = body_elem.find_all('div')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [10]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()
    

In [11]:
df = pd.DataFrame(results)
print(len(results))
df.head(5)

9


,title,author,category,date,content,region,link
0,Dibutuhkan Rp1 triliun untuk bangun flyover di...,M Fikri Setiawan,Category not found,"Rabu, 27 September 2023 8:30 WIB",Content not found,banten,https://jabar.antaranews.com/berita/470544/dib...
1,Pemprov Jabar terus mengupayakan basmi NII Pan...,Ricky Prayoga,Category not found,"Selasa, 26 September 2023 23:00 WIB",12Tampilkan Semua\n,banten,https://jabar.antaranews.com/berita/470523/pem...
2,Selatan Jawa dan sejumlah wilayah berpotensi a...,Zubi Mahrofi,Category not found,"Rabu, 27 September 2023 8:47 WIB",Content not found,banten,https://jabar.antaranews.com/berita/470562/sel...
3,Dinas Pertanian Garut minta masyarakat terapka...,Feri Purnama,Category not found,"Selasa, 26 September 2023 23:18 WIB",123Tampilkan Semua\n,banten,https://jabar.antaranews.com/berita/470520/din...
4,BMKG imbau masyarakat selatan Jawa waspada pot...,Zubi Mahrofi,Category not found,"Rabu, 27 September 2023 8:51 WIB",Content not found,banten,https://jabar.antaranews.com/berita/470568/bmk...


In [ ]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'./excel/jabar_antranews_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')